In [3]:
import requests
import random
import time
from datetime import datetime, timezone

def get_latest_block_hash():
    url = "https://blockstream.info/api/blocks/tip/hash"
    res = requests.get(url)
    return res.text if res.status_code == 200 else None

def get_block_info(block_hash):
    url = f"https://blockstream.info/api/block/{block_hash}"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else None

def get_block_txids(block_hash):
    url = f"https://blockstream.info/api/block/{block_hash}/txids"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else []

def get_prev_block_hash(block_hash):
    info = get_block_info(block_hash)
    if info and "prev_block" in info:
        return info["prev_block"]
    return None

def get_transaction_details(txid):
    url = f"https://blockstream.info/api/tx/{txid}"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else None

def satoshi_to_btc(sat):
    return sat / 1e8

# メイン処理
print("🔄 過去2日間のブロックから日時の異なる2件を取得中...")

block_hash = get_latest_block_hash()
tx_samples = []
seen_timekeys = set()
target_count = 2
max_blocks = 288

for _ in range(max_blocks):
    if not block_hash:
        break

    block_info = get_block_info(block_hash)
    if not block_info or "timestamp" not in block_info:
        block_hash = get_prev_block_hash(block_hash)
        continue

    timestamp = block_info["timestamp"]
    dt = datetime.fromtimestamp(timestamp, tz=timezone.utc)

    # ✅ 10分単位で時間キーを作成して、重複を避ける
    minute_bucket = (dt.minute // 10) * 10
    ts_key = dt.strftime('%Y-%m-%d %H:') + str(minute_bucket).zfill(2)

    if ts_key in seen_timekeys:
        block_hash = get_prev_block_hash(block_hash)
        time.sleep(0.05)
        continue

    txids = get_block_txids(block_hash)
    if txids:
        txid = random.choice(txids)
        tx = get_transaction_details(txid)
        if tx:
            tx_samples.append((dt, tx))
            seen_timekeys.add(ts_key)

    if len(tx_samples) >= target_count:
        break

    block_hash = get_prev_block_hash(block_hash)
    time.sleep(0.05)

# 出力
print(f"✅ {len(tx_samples)}件のトランザクションを取得しました。\n")

for i, (dt, tx) in enumerate(tx_samples):
    from_addresses = []
    for vin in tx['vin']:
        if 'prevout' in vin and vin['prevout'] and 'scriptpubkey_address' in vin['prevout']:
            from_addresses.append(vin['prevout']['scriptpubkey_address'])

    to_addresses = []
    total_output = 0
    for vout in tx['vout']:
        if 'scriptpubkey_address' in vout:
            to_addresses.append(vout['scriptpubkey_address'])
            total_output += vout['value']

    print(f"🧾 トランザクション {i+1}: {tx['txid']}")
    print("  📅 日時（UTC）:", dt.strftime('%Y-%m-%d %H:%M:%S'))
    print("  🔸送信元:", from_addresses if from_addresses else "（コインベース or 不明）")
    print("  🔹送信先:", to_addresses)
    print("  💰総出力金額:", satoshi_to_btc(total_output), "BTC")
    print("  💸手数料:", satoshi_to_btc(tx['fee']), "BTC")
    print("  ⏱サイズ:", tx['size'], "bytes\n")


🔄 過去2日間のブロックから日時の異なる2件を取得中...
✅ 1件のトランザクションを取得しました。

🧾 トランザクション 1: 1d7774d777086a114c2403e426b8e103b098d10f69e4e4063aae8facda3ace58
  📅 日時（UTC）: 2025-06-11 05:00:01
  🔸送信元: ['bc1qsml5hv6km620gkf35v7346zqtk7enmpqzk6h2p']
  🔹送信先: ['bc1qsml5hv6km620gkf35v7346zqtk7enmpqzk6h2p']
  💰総出力金額: 1.839e-05 BTC
  💸手数料: 1.5e-06 BTC
  ⏱サイズ: 229 bytes

